In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

In [ ]:
from sklearn.preprocessing import LabelEncoder, scale, StandardScaler, Normalizer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.base import BaseEstimator,TransformerMixin,RegressorMixin

In [ ]:
from catboost import CatBoostClassifier, Pool

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.over_sampling import RandomOverSampler

In [ ]:
from xgboost import XGBClassifier

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
from scipy.special import boxcox1p

Read files

In [ ]:
car_train=pd.read_csv("../data/train.csv")
car_test=pd.read_csv("../data/test.csv")

# Modelos

In [ ]:
from sklearn.metrics import r2_score, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.svm import SVC

from sklearn.linear_model import LogisticRegression

In [ ]:
def classifier_models(X_train, X_test, y_train, y_test, scoring, N_SPLITS):

    # KNeighborsClassifier
    print("KNeighborsClassifier\n")
    neigh = KNeighborsClassifier()
    model_report(neigh, X_train, X_test, y_train, y_test, scoring, N_SPLITS)
    
    # Decision Tree
    print("\nDecisionTreeClassifier\n")
    dec_tree = DecisionTreeClassifier(random_state=42)
    model_report(dec_tree, X_train, X_test, y_train, y_test, scoring, N_SPLITS)
    
    # RandomForestClassifier
    print("\nRandomForestClassifier\n")
    rf_clf = RandomForestClassifier(random_state=42)
    model_report(rf_clf, X_train, X_test, y_train, y_test, scoring, N_SPLITS)
    
    # ExtraTreesClassifier
    print("\nExtraTreesClassifier\n")
    et_clf = ExtraTreesClassifier(random_state=42)
    model_report(et_clf, X_train, X_test, y_train, y_test, scoring, N_SPLITS)
    
    # XGBoost
    print("\nXGBClassifier\n")
    xgboost = XGBClassifier(random_state = 42)
    model_report(xgboost, X_train, X_test, y_train, y_test, scoring, N_SPLITS)
    
    # LGBMClassifier
    print("\nLGBMClassifier\n")
    lgbm = LGBMClassifier()
    model_report(lgbm, X_train, X_test, y_train, y_test, scoring, N_SPLITS)

In [ ]:
def svm_models(X_train, X_test, y_train, y_test, scoring, N_SPLITS):
    print("\nSVM\n")
    print("\nlinear\n")
    svc_linear = SVC(random_state=42, kernel = "linear")
    model_report(svc_linear, X_train, X_test, y_train, y_test, scoring, N_SPLITS)
    
    print("\npoly\n")
    svc_poly = SVC(random_state=42, kernel = "poly")
    model_report(svc_poly, X_train, X_test, y_train, y_test, scoring, N_SPLITS)
    
    print("\nrbf\n")
    svc_rbf = SVC(random_state=42, kernel = "rbf")
    model_report(svc_rbf, X_train, X_test, y_train, y_test, scoring, N_SPLITS)
    
    print("\nsigmoid\n")
    svc_sigmoid = SVC(random_state=42, kernel = "sigmoid")
    model_report(svc_sigmoid, X_train, X_test, y_train, y_test, scoring, N_SPLITS)

In [ ]:
def model_report(model, X_train, X_test, y_train, y_test, scoring, splits):
    print("\nCROSS VALIDATION SCORES\n")
    model_score = cross_val_score(model, X_train, y_train, scoring = scoring, cv=splits)
    print("cross_val_score " + scoring, model_score.mean(), model_score.std())
    model_score = cross_val_score(model, X_train, y_train, scoring = "f1_macro", cv=splits)
    print("cross_val_score f1 macro ", model_score.mean(), model_score.std())
    model_score = cross_val_score(model, X_train, y_train, scoring = "f1_micro", cv=splits)
    print("cross_val_score f1 micro ", model_score.mean(), model_score.std())
    model_score = cross_val_score(model, X_train, y_train, scoring = "precision_macro", cv=splits)
    print("cross_val_score precision macro ", model_score.mean(), model_score.std())
    model_score = cross_val_score(model, X_train, y_train, scoring = "precision_micro", cv=splits)
    print("cross_val_score precision micro", model_score.mean(), model_score.std())
    model_score = cross_val_score(model, X_train, y_train, scoring = "recall_macro", cv=splits)
    print("cross_val_score recall macro ", model_score.mean(), model_score.std())
    model_score = cross_val_score(model, X_train, y_train, scoring = "recall_micro", cv=splits)
    print("cross_val_score recall macro ", model_score.mean(), model_score.std())
    
    
#    model.fit(X_train, y_train, eval_metric = "mlogloss", eval_set = [(X_test, y_test)])
    model.fit(X_train, y_train)
    y_pred= model.predict(X_test)
    print("\nAccuracy on Training set: ", model.score(X_train,y_train))
    print("Accuracy on Testing set: ",model.score(X_test,y_test))
    error_table(X_train, X_test, y_train, y_test, y_pred)

In [ ]:
def error_table(X_train, X_test, y_train, y_test, y_pred):
    print("\nERROR TABLE")
    print('accuracy_score        : ', accuracy_score(y_test, y_pred))
    print('f1_score macro        : ', f1_score(y_test, y_pred, average='macro'))
    print('f1_score micro        : ', f1_score(y_test, y_pred, average='micro'))
    print('precision_score macro : ', precision_score(y_test, y_pred, average='macro'))
    print('precision_score micro : ', precision_score(y_test, y_pred, average='micro'))
    print('recall_score macro.   : ', recall_score(y_test, y_pred, average='macro'))
    print('recall_score micro.   : ', recall_score(y_test, y_pred, average='micro'))

In [ ]:
def best_tree(X_train, y_train):
    #Randomized Search CV

    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree 3-10 to deal with overfitting
    max_depth = [3, 4, 5, 6, 7, 8, 9]
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10, 15, 100]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 5, 10]
    
    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf}
    
    # search across different combinations
    rf_random = RandomizedSearchCV(estimator = RandomForestClassifier(), 
                                   param_distributions = random_grid, 
                                   scoring=scoring, 
                                   n_iter = 15, cv = 5, verbose=2, random_state=42, n_jobs = 1)
    
    rf_random.fit(X_train,y_train)
    
    return rf_random

In [ ]:
def best_svc(X_train_standard, y_train):
    # Create the random grid
    random_grid = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
               'gamma': ['scale', 'auto'],
               'C': [0.1,1, 10, 100]}
    
    # search across 100 different combinations
    svc_random = RandomizedSearchCV(estimator = SVC(), 
                               param_distributions = random_grid, 
                               scoring=scoring, 
                               n_iter = 15, cv = 5, verbose=2, random_state=42, n_jobs = 1)
    svc_random.fit(X_train_standard, y_train)
    
    return svc_random

In [ ]:
def best_extra_tree(X_train, y_train):
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt', 'log2']
    # Maximum number of levels in tree
    max_depth = [3, 4, 5, 6, 7, 8, 9]
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10, 15, 100]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 5, 10]
    
    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf}
    
    # search across 100 different combinations
    extra_tree_random = RandomizedSearchCV(estimator = ExtraTreesClassifier(), 
                               param_distributions = random_grid, 
                               scoring=scoring, 
                               n_iter = 15, cv = 5, verbose=2, random_state=42, n_jobs = 1)
    extra_tree_random.fit(X_train, y_train)
    
    return extra_tree_random

In [ ]:
def best_xgboost(X_train, X_test, y_train, y_test):
    
    param_dist = {'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 1000, num = 12)],
# the learning rate of our GBM (i.e. how much we update our prediction with each successive tree); eta.
# Lower values avoid over-fitting.                  
              'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5],
              'subsample': [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.],
# the maximum depth of a tree; max_depth. Lower values avoid over-fitting                  
              'max_depth': [3, 4, 5, 6, 7, 8],
# the ratio of features used (i.e. columns used); colsample_bytree. Lower ratios avoid over-fitting.
              'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1.],
# the minimum sum of instance weight needed in a leaf, in certain applications this relates directly to 
# the minimum number of instances needed in a node; min_child_weight. Larger values avoid over-fitting.                  
              'min_child_weight': [1, 5, 15, 20, 50]
             }
    
    print(param_dist)

    xgboost_random = RandomizedSearchCV(estimator = XGBClassifier(), 
                               param_distributions = param_dist, 
                               scoring=scoring, 
                               n_iter = 15, cv = 5, verbose=2, random_state=42, n_jobs = 1)
    
    xgboost_random.fit(X_train, y_train, eval_metric = "mlogloss", eval_set = [(X_test, y_test)])
    
    return xgboost_random

In [ ]:
def best_lgbm(X_train, X_test):
    param_dist = {
        'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 1000, num = 12)],
        'learning_rate':  [0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5],
        'max_depth': [3, 4, 5, 6, 7, 8, 9],
        'min_data_in_leaf': [int(x) for x in np.linspace(start = 100, stop = 1000, num = 12)],
        'num_leaves': [int(x) for x in np.linspace(start = 100, stop = 1000, num = 12)]
    }
    
    print(param_dist)
    
    lgbm_random = RandomizedSearchCV(estimator = LGBMClassifier(), 
                               param_distributions = param_dist, 
                               scoring=scoring, 
                               n_iter = 15, cv = 5, verbose=2, random_state=42, n_jobs = 1)
    
    lgbm_random.fit(X_train, y_train)
    
    return lgbm_random

In [ ]:
def best_catboost(X_train, y_train):
    
    grid = {
        'iterations': [800, 1000, 1200], 
        'learning_rate': [0.03, 0.06, 0.1],
        'random_strength': [0.03, 0.06, 0.1],
        'depth': [4, 5, 6, 7, 8, 9, 10],
        'one_hot_max_size': [2,3,4,5,6,7,8],
        'od_type': ['IncToDec', 'Iter']}
    
    model = CatBoostClassifier(loss_function='MultiClass', eval_metric='Accuracy', leaf_estimation_method='Newton')
    
    randomized_search_result = model.randomized_search(grid,
                                                       X=X_train,
                                                       y=y_train,
                                                       cv=5, n_iter=15, plot = True)
                                                       # plot = True, verbose = True
    
    return randomized_search_result

* [Version 1](#section-1)
* [Version 2](#section-2)

In [ ]:
car_train.shape

In [ ]:
car_test.shape

In [ ]:
car_train.head()

In [ ]:
car_train.info()

In [ ]:
car_test.head()

In [ ]:
car_test.info()

## Missing values

In [ ]:
#df_train.loc[df_train['brand_name'] == 'Maruti']['Asientos'].mode()[0]
def fill_na_with_mode(ds, brandname, column):
    if ds.loc[ds['Company'] == brandname][column].isnull().any() == False:
        fill_value = ds.loc[ds['Company'] == brandname][column].mode()[0]
        condit = ((ds['Company'] == brandname) & (ds[column].isnull()))
        ds.loc[condit, column] = ds.loc[condit, column].fillna(fill_value)

In [ ]:
def miss_perfect(dataframe):
    miss_percent = (dataframe.isnull().sum() / len(dataframe)) * 100
    missing = pd.DataFrame({"percent": miss_percent, 'count': dataframe.isnull().sum()}).sort_values(
        by="percent", ascending=False)
    return missing.loc[missing['percent'] > 0]

In [ ]:
miss_perfect(car_train)

In [ ]:
miss_perfect(car_test)

In [ ]:
print("Valores únicos en el conjunto de entrenamiento:\n")
for i in car_train.columns:
    print(i, car_train[i].unique())

In [ ]:
print("Valores únicos en el conjunto de test:\n")
for i in car_test.columns:
    print(i, car_test[i].unique())

In [ ]:
car_train.describe()

In [ ]:
print("There is {} duplicated values in data frame".format(car_train.duplicated().sum()))

In [ ]:
print("There is {} duplicated values in data frame".format(car_test.duplicated().sum()))

In [ ]:
car_test.describe()

## EDA

In [ ]:
car_train.hist(bins='auto', figsize=(18, 22), layout=(5, 2))
plt.savefig("./figures/hist.png")
plt.show()

In [ ]:
sns.barplot(car_train["Mano"], car_train["Kilometros"],hue= car_train["Tipo_marchas"],palette="spring")
plt.xticks(rotation=80)
plt.title("Comparación Tipo marchas y Kilometros")

plt.savefig("./figures/tipo_marchas_km.png")
plt.show()

In [ ]:
sns.barplot(car_train["Mano"], car_train["Kilometros"],hue= car_train["Ciudad"],palette="spring")
plt.xticks(rotation=80)
plt.title("Comparación Ciudad y Kilómetros")

plt.savefig("./figures/ciudad_km.png")
plt.show()

## Variables continuas

In [ ]:
# Precio
plt.subplot(121)
sns.distplot(car_train['Precio_cat'],color="r");

plt.subplot(122)
car_train['Precio_cat'].plot.box(figsize=(16,5))

plt.savefig("./figures/precio_cat.png")
plt.show()

## Variables Discretas

In [ ]:
sns_plot = sns.countplot(y='Ciudad',data=car_train)
fig = sns_plot.get_figure()
fig.savefig("./figures/ciudad.png")
print(car_train.Ciudad.value_counts(normalize=True)*100)

In [ ]:
sns_plot = sns.countplot(x='Mano',data=car_train)
fig = sns_plot.get_figure()
fig.savefig("./figures/mano.png")
print(car_train.Mano.value_counts(normalize=True)*100)

In [ ]:
sns_plot = sns.countplot(x='Asientos',data=car_train)
fig = sns_plot.get_figure()
fig.savefig("./figures/asientos.png")
print(car_train.Asientos.value_counts())

In [ ]:
sns_plot = sns.countplot(x='Año',data=car_train)
plt.xticks(rotation='vertical')
fig = sns_plot.get_figure()
fig.savefig("./figures/anio.png")
print(car_train.Año.value_counts())

In [ ]:
sns_plot = sns.countplot(y="Motor_CC", data=car_train, palette="Greens_d",
              order=car_train.Motor_CC.value_counts().iloc[:10].index)
fig = sns_plot.get_figure()
fig.savefig("./figures/motor_cc.png")

In [ ]:
sns_plot = sns.countplot(y='Combustible',data=car_train)
fig = sns_plot.get_figure()
fig.savefig("./figures/combustible.png")
print(car_train.Combustible.value_counts(normalize=True)*100)

## Comparaciones con Precio_cat

In [ ]:
sns_plot = sns.barplot(y="Ciudad", x="Precio_cat", data=car_train)
fig = sns_plot.get_figure()
fig.savefig("./figures/ciudad_precio_cat.png")

In [ ]:
fig_dims = (20, 30)
fig, ax = plt.subplots(figsize=fig_dims)
sns.boxplot(x="Precio_cat", y="Ciudad", data=car_train )

In [ ]:
print(car_train.Asientos.value_counts())
sns_plot = sns.barplot(x='Asientos',y='Precio_cat',data=car_train)
fig = sns_plot.get_figure()
fig.savefig("./figures/asientos_precio_cat.png")

In [ ]:
sns_plot = sns.barplot(y="Precio_cat", x="Año", data=car_train )
plt.xticks(rotation='vertical')
fig = sns_plot.get_figure()
fig.savefig("./figures/anio_precio_cat.png")

In [ ]:
sns_plot = sns.barplot(y="Precio_cat", x="Mano", data=car_train)
fig = sns_plot.get_figure()
fig.savefig("./figures/mano_precio_cat.png")

In [ ]:
sns_plot = sns.barplot(y="Precio_cat", x="Combustible", data=car_train )
fig = sns_plot.get_figure()
fig.savefig("./figures/combustible_precio_cat.png")

<a id="section-1"></a>
# Versión Vanilla
Eliminamos todos los nulos, etiquetamos nuestros atributos y aplicamos distintos modelos

In [ ]:
car_train_1 = car_train.copy()
car_test_1 = car_test.copy()

In [ ]:
car_train_1 = car_train_1.drop(["Descuento", "id"], axis = 1)
car_test_1 = car_test_1.drop(["Descuento"], axis = 1)

In [ ]:
car_train_1.head()

In [ ]:
miss_perfect(car_train_1)

### Sustituir por la moda de la misma compañia 

In [ ]:
car_train_1 = car_train_1[car_train_1['Nombre'].notna()]
car_train_1.reset_index(drop=True)

In [ ]:
car_train_1.Consumo = car_train.Consumo.replace('0.0 kmpl', np.nan)

In [ ]:
part = car_train_1['Nombre'].astype(str).str.partition()
car_train_1['Company'] = part[0].str.title()
#car_train_2['Car_name'] = part[2]

### Comparar precio_cat con marcas

In [ ]:
sns_plot = sns.countplot(y='Company',data=car_train_1)
fig = sns_plot.get_figure()
fig.savefig("./figures/company.png")
print(car_train_1.Company.value_counts(normalize=True)*100)

In [ ]:
fig_dims = (20, 25)
fig, ax = plt.subplots(figsize=fig_dims)
sns_plot = sns.barplot(y="Company", x="Precio_cat", data=car_train_1)
fig = sns_plot.get_figure()
fig.savefig("./figures/company_precio_cat.png")

In [ ]:
for col in list(car_train_1.columns):
    if car_train_1[col].isnull().any():
         for brand in list(car_train_1.Company.unique()):
                fill_na_with_mode(car_train_1, brand, col)

In [ ]:
car_train_1 = car_train_1.drop(["Company"], axis = 1)

### Or fill the missing values with the mean if numeric or the mode if object

In [ ]:
for i in car_train_1.columns:
    if car_train_1[i].isnull().values.any():
        if car_train_1[i].dtype == "O":
            car_train_1[i]= car_train_1[i].fillna((car_train_1[i].mode()[0]))
        else:
            car_train_1[i]= car_train_1[i].fillna((car_train_1[i].mean()))

### Or just drop all the nan

In [ ]:
car_train_1 = car_train_1.dropna()

In [ ]:
car_train_1_orig = car_train_1.copy()
car_test_1_orig = car_test_1.copy()

## Etiquetar atributos

In [ ]:
for i in list(car_train_1.columns):
    if car_train_1[i].dtypes == "object":
        to_read = "../data/" + i.lower() + ".csv"
        label = LabelEncoder().fit(pd.read_csv(to_read)[i])
        car_train_1[i] = label.transform(car_train_1_orig[i])
        car_test_1[i] = label.transform(car_test_1_orig[i])

In [ ]:
correcto = True

for asientos in car_train_1_orig.Asientos.unique():
    label_train = car_train_1.Asientos[car_train_1_orig.Asientos == asientos].unique()
    label_test = car_test_1.Asientos[car_test_1_orig.Asientos == asientos].unique()
    
    if len(label_train)==0 or len(label_test)==0:
        continue
    
    if label_train != label_test:
        print("Error, {} es etiquetado en train como {} y en test como {}".format(asientos, label_train, label_test))
        correcto = False
        break
        
if correcto:
    print("Todo perfecto")

## Modelos

In [ ]:
scoring = "accuracy"
N_SPLITS = 5

In [ ]:
y = car_train_1['Precio_cat']
X = car_train_1.drop(['Precio_cat'],axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= 42, test_size=0.2)

In [ ]:
classifier_models(X_train, X_test, y_train, y_test, scoring, N_SPLITS)

In [ ]:
Xo, yo = SMOTE(random_state = 42).fit_resample(X, y)
print(Counter(yo))
X_train, X_test, y_train, y_test = train_test_split(Xo, yo, random_state= 42, test_size=0.2)

In [ ]:
classifier_models(X_train, X_test, y_train, y_test, scoring, N_SPLITS)

In [ ]:
Xo, yo = RandomOverSampler(random_state = 42).fit_resample(X, y)
print(Counter(yo))
X_train, X_test, y_train, y_test = train_test_split(Xo, yo, random_state= 42, test_size=0.2)

In [ ]:
classifier_models(X_train, X_test, y_train, y_test, scoring, N_SPLITS)

### With standardization 

In [ ]:
sc = StandardScaler()
X_train_standard = sc.fit_transform(X_train)
X_test_standard = sc.transform(X_test)

In [ ]:
classifier_models(X_train_standard, X_test_standard, y_train, y_test, scoring, N_SPLITS)

In [ ]:
svm_models(X_train_standard, X_test_standard, y_train, y_test, scoring, N_SPLITS)

### Normalizer

In [ ]:
norm = Normalizer()
X_train_norm = norm.fit_transform(X_train)
X_test_norm = norm.transform(X_test)

In [ ]:
classifier_models(X_train_norm, X_test_norm, y_train, y_test, scoring, N_SPLITS)

## Tuning Best Models

### Random Forest

In [ ]:
best_rfc_1 = best_tree(X_train, y_train)

In [ ]:
best_rfc_1.best_params_

In [ ]:
best_rfc_1 = RandomForestClassifier(max_depth = 9, max_features = 'auto', min_samples_leaf = 1,
                                  min_samples_split = 15, n_estimators = 1000, random_state = 42)

In [ ]:
model_report(best_rfc_1, X_train, X_test, y_train, y_test, scoring, N_SPLITS)

In [ ]:
plt.figure(figsize = (12,8))
feat_importances = pd.Series(best_rfc_1.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh')
plt.savefig("./figures/feature_importances_rfc_1.png")
plt.show()

### SVC

In [ ]:
best_svc_1 = best_svc(X_train_standard, y_train)

In [ ]:
best_svc_1.best_params_

In [ ]:
best_svc_1 = SVC(C = 100, gamma = 'auto', kernel = 'rbf', random_state = 42)

In [ ]:
model_report(best_svc_1, X_train_standard, X_test_standard, y_train, y_test, scoring, N_SPLITS)

### Best Extra tree

In [ ]:
best_extra_tree_1 = best_tree(X_train, y_train)

In [ ]:
best_extra_tree_1.best_params_

In [ ]:
best_extra_tree_1 = ExtraTreesClassifier(n_estimators = 400, min_samples_split = 15,
                                        min_samples_leaf = 1, max_features = 'auto', max_depth = 9)

In [ ]:
model_report(best_extra_tree_1, X_train, X_test, y_train, y_test, scoring, N_SPLITS)

In [ ]:
plt.figure(figsize = (12,8))
feat_importances = pd.Series(best_extra_tree_1.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh')
plt.show()

### Cat boost

In [ ]:
catboost_1 = best_catboost(X_train, y_train)

In [ ]:
catboost_1['params']

In [ ]:
catboost_1 = CatBoostClassifier(
    depth = 10,
    one_hot_max_size = 2,
    iterations = 800,
    random_strength = 0.03,
    learning_rate = 0.1,
    od_type = 'IncToDec',
    loss_function='MultiClass',
    eval_metric='Accuracy',
    leaf_estimation_method='Newton',
    verbose=0)

In [ ]:
model_report(catboost_1, X_train, X_test, y_train, y_test, scoring, N_SPLITS)

In [ ]:
catboost_1 = CatBoostClassifier(
    depth = 7,
    iterations = 1000,
    random_strength = 0.1,
    learning_rate = 0.1,
    loss_function='MultiClass',
    eval_metric='Accuracy',
    leaf_estimation_method='Newton',
    verbose=0)

In [ ]:
model_report(catboost_1, X_train, X_test, y_train, y_test, scoring, N_SPLITS)

### XGBoost

In [ ]:
best_xgboost_1 = best_xgboost(X_train, X_test, y_train, y_test)

In [ ]:
best_xgboost_1.best_params_

In [ ]:
best_xgboost_1 = XGBClassifier(subsample = 0.7, n_estimators = 918, min_child_weight = 1, max_depth = 8,
                               learning_rate = 0.15, colsample_bytree = 1.0)

Subsample ratio of the training instances. Setting it to 0.5 means that XGBoost would randomly sample half of the training data prior to growing trees. and this will prevent overfitting. Subsampling will occur once in every boosting iteration.

In [ ]:
model_report(best_xgboost_1, X_train, X_test, y_train, y_test, scoring, N_SPLITS)

### LGBM 

In [ ]:
best_lgbm_1 = best_lgbm(X_train, y_train)

In [ ]:
best_lgbm_1.best_params_

In [ ]:
best_lgbm_1 = LGBMClassifier(num_leaves = 754, n_estimators = 509, min_data_in_leaf = 509, max_depth = 5,
                             learning_rate = 0.4, verbosity = 4)

In [ ]:
model_report(best_lgbm_1, X_train, X_test, y_train, y_test, scoring, N_SPLITS)

In [ ]:
plt.figure(figsize = (12,8))
feat_importances = pd.Series(best_lgbm_1.feature_importances_, index = X.columns)
feat_importances.nlargest(20).plot(kind='barh')
plt.show()

## Generate file Version 1

In [ ]:
tests_ids_1 = car_test_1['id']
del car_test_1['id']

In [ ]:
predict_1 = best_xgboost_1.predict(car_test_1)
# Generamos 
df_result = pd.DataFrame({'id': tests_ids_1, 'Precio_cat': predict_1 })
df_result.to_csv("mis_resultados_xgboost_1.csv", index=False)

In [ ]:
print(predict_1)

In [ ]:
predict_1 = best_rfc_1.predict(car_test_1)
submit = pd.read_csv('../data/sample.csv')
submit['Precio_cat'] = predict_1
submit.to_csv('mis_resultados_rfc_1.csv', index=False)

In [ ]:
print(predict_1)

In [ ]:
predict_1 = catboost_1.predict(car_test_1)

In [ ]:
print(predict_1.ravel())

In [ ]:
submit = pd.read_csv('../data/sample.csv')
submit['Precio_cat'] = predict_1.ravel()
submit.to_csv('mis_resultados_catboost_1.csv', index=False)

In [ ]:
scaled_test_1 = sc.transform(car_test_1)

In [ ]:
predict_1 = best_svc_1.predict(scaled_test_1)

In [ ]:
print(predict_1)

In [ ]:
submit = pd.read_csv('../data/sample.csv')
submit['Precio_cat'] = predict_1
submit.to_csv('mis_resultados_svc_1.csv', index=False)

<a id="section-2"></a>
# Versión con preprocesado y más técnicas
Hacemos EDA, trabajamos con outliers, skewed data, imbalanced dataset, distintos tipos de encoding (label encoding y one hot encoding)

In [ ]:
car_train_2 = car_train.copy()
car_test_2 = car_test.copy()

In [ ]:
#since the discount column is missing in both train and test set lets remove that column
car_train_2=car_train_2.drop(['Descuento'],axis=1)

### Fill missing values with the mode of the company brand

In [ ]:
car_train_2 = car_train_2[car_train_2['Nombre'].notna()]
car_train_2 = car_train_2.reset_index(drop=True)

In [ ]:
part = car_train_2['Nombre'].astype(str).str.partition()
car_train_2['Company'] = part[0].str.title()
#car_train_2['Car_name'] = part[2]

In [ ]:
for col in list(car_train_2.columns):
    if car_train_2[col].isnull().any():
         for brand in list(car_train_2.Company.unique()):
                fill_na_with_mode(car_train_2, brand, col)

In [ ]:
car_train_2.drop(['Company'], axis = 1, inplace = True)

In [ ]:
car_train_2.head()

In [ ]:
car_train_2 = car_train_2.dropna()

### or fill mising values with mode or mean

In [ ]:
for i in car_train_2.columns:
    if car_train_2[i].isnull().values.any():
        if car_train_2[i].dtype == "O":
            car_train_2[i]= car_train_2[i].fillna((car_train_2[i].mode()[0]))
        else:
            car_train_2[i]= car_train_2[i].fillna((car_train_2[i].mean()))

### or drop all

In [ ]:
car_train_2 = car_train_2.dropna()

### Preprocessing

In [ ]:
car_train_2['Consumo']=car_train_2.Consumo.astype(str).str.partition()[0].astype('float')
car_train_2['Motor_CC']=car_train_2.Motor_CC.astype(str).str.partition()[0].astype('float')
car_train_2['Potencia']=car_train_2.Potencia.astype(str).str.partition()[0].astype('float')

In [ ]:
# Consumo
plt.subplot(121)
sns.distplot(car_train_2['Consumo'],color="b");

plt.subplot(122)
car_train_2['Consumo'].plot.box(figsize=(16,5))

plt.savefig("./figures/consumo_dis.png")
plt.show()

In [ ]:
# Motor CC
plt.subplot(121)
sns.distplot(car_train_2['Motor_CC'],color="g");

plt.subplot(122)
car_train_2['Motor_CC'].plot.box(figsize=(16,5))

plt.savefig("./figures/motor_dis.png")
plt.show() 

In [ ]:
# Potencia
plt.subplot(121)
sns.distplot(car_train_2['Potencia'],color="m");

plt.subplot(122)
car_train_2['Potencia'].plot.box(figsize=(16,5))

plt.savefig("./figures/potencia_dis.png")
plt.show() 

### Outliers

In [ ]:
features_outliers=['Kilometros', 'Potencia','Consumo','Motor_CC']

In [ ]:
for i in features_outliers:
    fig, ax = plt.subplots()
    fig.set_size_inches(12,6)
    #plt.xlim(-10,10)
    sns_plot = sns.boxplot(x=i,data=car_train_2,ax=ax)
    fig = sns_plot.get_figure()
    fig.savefig("./figures/outlier_" + i + ".png")  

In [ ]:
for feature in features_outliers:
    print(feature)
    print('max threshold :',car_train_2[feature].quantile(0.999))
    print('min threshold :',car_train_2[feature].quantile(0.001))

In [ ]:
class RemoveOutliers(BaseEstimator,TransformerMixin):
    """This class removes outliers from data.
    Note: Outlier values are hard coded
    """

    def fit (self,X,y=None):
        return self
    
    
    def transform(self,X,y=None):
        X=X[X['Kilometros'] < 262000] 
            
        X=X[X['Consumo'] > 0.0]
        
        X=X[X['Potencia'] <= 530]
        
        X=X[X['Motor_CC'] <= 5900 ]
        
        return X

In [ ]:
car_train_2 = RemoveOutliers().fit_transform(car_train_2)

In [ ]:
for i in features_outliers:
    fig, ax = plt.subplots()
    fig.set_size_inches(12,6)
    #plt.xlim(-10,10)
    sns_plot = sns.boxplot(x=i,data=car_train_2,ax=ax)
    fig = sns_plot.get_figure()
    fig.savefig("./figures/no_outlier_" + i + ".png")  

### Skewed data

In [ ]:
car_train_2[features_outliers].hist(bins=50,figsize=(15,10))

plt.savefig("./figures/skewed_data.png")

In [ ]:
skewness=[]
for i in features_outliers:
    skewness.append(car_train_2[i].skew())
pd.DataFrame(data=skewness,index=features_outliers,columns=['skewness']).sort_values(by='skewness',ascending=False)

In [ ]:
pd.DataFrame(data=skewness,index=features_outliers,columns=['skewness']).sort_values(by='skewness',ascending=False).to_latex()

In [ ]:
class FeaturesTransformer(BaseEstimator,TransformerMixin):
    """This class trnsforms numberical featuress in the dataset.
    Note: Transformations are hard coded.
    """
    def fit(self,X,y=None):
        return self
    def transform(self,X,y=None):
        from scipy.special import boxcox1p        
        X['Kilometros']=X['Kilometros'].apply(lambda x: boxcox1p(x,0.33))
        
        X['Potencia']=X['Potencia'].apply(lambda x:np.log(x) )
        
        X['Motor_CC']=X['Motor_CC'].apply(lambda x: np.log(x))
        
        X['Consumo']=X['Consumo'].apply(lambda x: np.log1p(x)**4)
        
        return X

In [ ]:
car_train_2 = FeaturesTransformer().fit_transform(car_train_2)

In [ ]:
car_train_2[features_outliers].hist(bins=50,figsize=(15,10))
plt.savefig("./figures/no_skewed_data.png")

In [ ]:
skewness=[]
for i in features_outliers:
    skewness.append(car_train_2[i].skew())
pd.DataFrame(data=skewness,index=features_outliers,columns=['skewness']).sort_values(by='skewness',ascending=False)

In [ ]:
pd.DataFrame(data=skewness,index=features_outliers,columns=['skewness']).sort_values(by='skewness',ascending=False).to_latex()

## Ordinal categorical data

In [ ]:
fig_dims = (15, 8)
fig, ax = plt.subplots(figsize = fig_dims)
sns.boxplot(y = "Precio_cat", x = "Mano", data = car_train_2 )

In [ ]:
car_train_2.replace({'First':1,'Second':2,'Third':3,'Fourth & Above':4},inplace = True)

In [ ]:
car_train_2_orig = car_train_2.copy()
car_test_2_orig = car_test_2.copy()

In [ ]:
car_train_2.dtypes

## Dummy variables

In [ ]:
tipo_marchas = car_train_2[['Tipo_marchas']]
tipo_marchas = pd.get_dummies(tipo_marchas, drop_first=True)

In [ ]:
ciudad = car_train_2[['Ciudad']]
ciudad = pd.get_dummies(ciudad, drop_first=True)

In [ ]:
combustible = car_train_2[['Combustible']]
combustible = pd.get_dummies(combustible, drop_first=True)

In [ ]:
car_train_2 = pd.concat([car_train_2, ciudad , tipo_marchas, combustible], axis = 1)

In [ ]:
car_train_2_orig = car_train_2.copy()
car_test_2_orig = car_test_2.copy()

In [ ]:
label = LabelEncoder().fit(pd.read_csv('../data/nombre.csv').Nombre)
car_train_2.Nombre = label.transform(car_train_2_orig.Nombre)
car_test_2.Nombre = label.transform(car_test_2_orig.Nombre)

In [ ]:
car_train_2.drop(['Ciudad', 'Combustible', 'Tipo_marchas'], axis = 1, inplace = True)

In [ ]:
car_train_2.head()

## Test data

In [ ]:
car_test_2 = car_test_2.drop('Descuento',axis=1)

In [ ]:
car_test_2['Consumo']=car_test_2.Consumo.astype(str).str.partition()[0].astype('float')
car_test_2['Motor_CC']=car_test_2.Motor_CC.astype(str).str.partition()[0].astype('float')
car_test_2['Potencia']=car_test_2.Potencia.astype(str).str.partition()[0].astype('float')

In [ ]:
car_test_2.replace({'First':1,'Second':2,'Third':3,'Fourth & Above':4},inplace = True)

In [ ]:
# Tipo_marchas
tipo_marchas = car_test_2[["Tipo_marchas"]]
tipo_marchas = pd.get_dummies(tipo_marchas,drop_first=True)

# ciudad
ciudad = car_test_2[["Ciudad"]]
ciudad = pd.get_dummies(ciudad,drop_first=True)

# Company
#company = car_test_2[["Company"]]
#company = pd.get_dummies(company ,drop_first=True)

# Combustible
combustible = car_test_2[["Combustible"]]
combustible = pd.get_dummies(combustible,drop_first=True)

In [ ]:
car_test_2 = pd.concat([car_test_2, ciudad, tipo_marchas, combustible ], axis = 1)

In [ ]:
car_test_2.drop(['Tipo_marchas', 'Ciudad', 'Combustible'], axis = 1, inplace = True)

In [ ]:
car_test_2.shape

In [ ]:
# Get missing columns in the training test
missing_cols = set( car_train_2.columns ) - set( car_test_2.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    car_test_2[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
car_test_2 = car_test_2[car_train_2.columns]

In [ ]:
car_test_2 = car_test_2.drop('Precio_cat', axis=1)

In [ ]:
car_train_2 = car_train_2.drop('id', axis=1)

In [ ]:
car_test_2.head()

## Construimos modelo

In [ ]:
scoring = "accuracy"
N_SPLITS = 5

In [ ]:
y = car_train_2['Precio_cat']
X = car_train_2.drop('Precio_cat', axis=1)

In [ ]:
Counter(y)

In [ ]:
Xo, yo = SMOTE(random_state = 42).fit_resample(X, y)
print(Counter(yo))
X_train, X_test, y_train, y_test = train_test_split(Xo, yo, random_state= 42, test_size=0.2)

In [ ]:
Xo, yo = RandomOverSampler(random_state = 42).fit_resample(X, y)
print(Counter(yo))
X_train, X_test, y_train, y_test = train_test_split(Xo, yo, random_state= 42, test_size=0.2)

In [ ]:
classifier_models(X_train, X_test, y_train, y_test, scoring, N_SPLITS)

In [ ]:
# StandardScaler
sc = StandardScaler()
X_train_standard = sc.fit_transform(X_train)
X_test_standard = sc.transform(X_test)

In [ ]:
classifier_models(X_train_standard, X_test_standard, y_train, y_test, scoring, N_SPLITS)

In [ ]:
svm_models(X_train_standard, X_test_standard, y_train, y_test, scoring, N_SPLITS)

Profundizamos sobre los que han conseguido mejores puntuaciones

### Best Random Forest Classifier

In [ ]:
best_rfc_2 = best_tree(X_train, y_train)

In [ ]:
best_rfc_2.best_params_

In [ ]:
best_rfc_2 = RandomForestClassifier(n_estimators = 400, min_samples_split = 15, 
                                  min_samples_leaf = 1, max_features = 'auto', max_depth = 9)

In [ ]:
model_report(best_rfc_2, X_train, X_test, y_train, y_test, scoring, N_SPLITS)

### Best SVC

In [ ]:
best_svc_2 = best_svc(X_train_standard, y_train)

In [ ]:
best_svc_2.best_params_

In [ ]:
best_svc_2 = SVC(kernel = 'rbf', gamma = 'auto', C = 100)

In [ ]:
model_report(best_svc_2, X_train_standard, X_test_standard, y_train, y_test, scoring, N_SPLITS)

### Best Extra tree

In [ ]:
best_extra_tree_2 = best_tree(X_train, y_train)

In [ ]:
best_extra_tree_2.best_params_

In [ ]:
best_extra_tree_2 = ExtraTreesClassifier(n_estimators = 400, min_samples_split = 15, min_samples_leaf = 1,
                                         max_features ='auto', max_depth = 9)

In [ ]:
model_report(best_extra_tree_2, X_train, X_test, y_train, y_test, scoring, N_SPLITS)

In [ ]:
plt.figure(figsize = (12,8))
feat_importances = pd.Series(best_extra_tree_2.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh')
plt.show()

### Catboost 

In [ ]:
catboost_2 = best_catboost(X_train, y_train)

In [ ]:
catboost_2['params']

In [ ]:
catboost_2 = CatBoostClassifier(
    iterations=800,
    learning_rate=0.1,
    one_hot_max_size = 4,
    random_strength=0.1,
    depth=9,
    od_type = 'IncToDec',
    loss_function='MultiClass',
    eval_metric='Accuracy',
    leaf_estimation_method='Newton',
    verbose = 0)

In [ ]:
model_report(catboost_2, X_train, X_test, y_train, y_test, scoring, N_SPLITS)

In [ ]:
catboost_2 = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    random_strength=0.1,
    depth=7,
    loss_function='MultiClass',
    eval_metric='Accuracy',
    leaf_estimation_method='Newton',
    verbose = 0)

In [ ]:
model_report(catboost_2, X_train, X_test, y_train, y_test, scoring, N_SPLITS)

### XGboost

In [ ]:
best_xgboost_2 = best_xgboost(X_train, X_test, y_train, y_test)

In [ ]:
best_xgboost_2.best_params_

In [ ]:
best_xgboost_2 = XGBClassifier(subsample = 0.7, n_estimators = 918, min_child_weight = 1,
                               max_depth = 8, learning_rate = 0.15, colsample_bytree = 1.0)

In [ ]:
model_report(best_xgboost_2, X_train, X_test, y_train, y_test, scoring, N_SPLITS)

### LGBM 

In [ ]:
best_lgbm_2 = best_lgbm(X_train, y_train)

In [ ]:
best_lgbm_2.best_params_

In [ ]:
best_lgbm_2 = LGBMClassifier(num_leaves = 754, n_estimators = 509, 
                             min_data_in_leaf = 509, max_depth = 5, learning_rate =0.4, verbose = -1)

In [ ]:
model_report(best_lgbm_2, X_train, X_test, y_train, y_test, scoring, N_SPLITS)

In [ ]:
plt.figure(figsize = (12,8))
feat_importances = pd.Series(best_lgbm_2.feature_importances_, index = X.columns)
feat_importances.nlargest(20).plot(kind='barh')
plt.show()

## Generando el fichero para Kaggle

In [ ]:
tests_ids_2 = car_test_2['id']
del car_test_2['id']

In [ ]:
predict_2 = best_xgboost_2.predict(car_test_2)
# Generamos 
df_result = pd.DataFrame({'id': tests_ids_2, 'Precio_cat': predict_2 })
df_result.to_csv("mis_resultados_xgboost_2.csv", index=False)

In [ ]:
print(predict_2)

In [ ]:
predict_2 = best_rfc_2.predict(car_test_2)
submit = pd.read_csv('../data/sample.csv')
submit['Precio_cat'] = predict_2
submit.to_csv('mis_resultados_rfc_2.csv', index=False)

In [ ]:
print(predict_2)

In [ ]:
predict_2 = catboost_2.predict(car_test_2)
submit = pd.read_csv('../data/sample.csv')
submit['Precio_cat'] = predict_2.ravel()
submit.to_csv('mis_resultados_catboost_2.csv', index=False)

In [ ]:
print(predict_2.ravel())

In [ ]:
scaled_test = sc.transform(car_test_2)

In [ ]:
predict_2 = best_svc_2.predict(scaled_test)
submit = pd.read_csv('../data/sample.csv')
submit['Precio_cat'] = predict_2
submit.to_csv('mis_resultados_svc_2.csv', index=False)

In [ ]:
print(predict_2)